# Front Matter

In [4]:
import tweepy 
import pandas as pd
pd.options.display.max_columns = None
pd.options.display.max_rows = 500
pd.set_option('display.max_colwidth', -1)
from identifiers_from_bio import *
import numpy as np
import umap
import spacy
from emoji import get_emoji_regexp
import altair as alt
from functools import partial
from sklearn.cluster import SpectralClustering, DBSCAN
alt.data_transformers.enable('json')

import fasttext
path_to_pretrained_model = 'lid.176.bin'
fmodel = fasttext.load_model(path_to_pretrained_model)


<ipython-input-4-1240720fdb18>:5: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


DataTransformerRegistry.enable('json')

In [57]:
filename = "panel_7_7_20.tsv"

# Read in the data

In [58]:
if 'json' in filename:
    df = [json.loads(x) for x in open(filename)]
    df = pd.DataFrame(df)
else:
    df = read_simple_user_info(filename)
    
df_len = len(df)
df = df[df.description != ""]
df = df[~df.description.isnull()]
df = df[~df.description.str.contains("(?i)(we are)|(not affiliated)")]
df_len_removed = len(df)

if 'status_lang' in df.columns:
    df = df[(df.status_lang.str.strip().isin(["en","","und","es"])) | df.protected]
elif 'lang' in df.columns:
    df =  df[(df.lang.isin(["en","","und","es"]))]

print(df_len, df_len_removed)
print(len(df))

/Users/kjoseph/opt/anaconda3/envs/myenv/lib/python3.8/site-packages/pandas/core/strings/accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


1646453 1191667
1168779


In [30]:
df.head(1)

,uid,name,screen_name,url,protected,location,description,followers_count,friends_count,created_at,utc_offset,time_zone,statuses_count,lang,status_created_at,status_coordinates,status_lang,profile_image_url_https,verified
1,2379457586,miguel andres,migue0225,None,False,NaN,"soy adicto al gym me gustan los manes que le guste ir al gym que sean activos cero pluma pruebo lo que sea cartagena ,quilla etc.",104,92,Sat Mar 08 23:40:35 +0000 2014,None,None,440,es,Mon Sep 01 19:06:27 +0000 2014,NaN,es,https://pbs.twimg.com/profile_images/452105301832265728/vULD4otO_normal.jpeg,False


In [31]:
word_idf, emoji_idf = count_phrases(df.description,
                                    n_cores=16,
                                    is_notebook=True)


100%|██████████| 517827/517827 [00:53<00:00, 9677.31it/s] 


In [32]:
# final cleaning
word_idf = word_idf.assign(phrase = word_idf.phrase.str.replace("_"," "))
word_idf = word_idf.groupby("phrase").agg({"n" :"sum"}).reset_index()
word_idf['n_token'] = word_idf['phrase'].map(lambda x: len(x.split(' ')))

In [33]:
word_idf.to_csv("wordidf_{}.csv".format(filename.replace(".tsv","")),index=False)

In [34]:
# Analysis in new_analysis.R 

In [35]:
vocab = word_idf[(word_idf.n_token < 4)].phrase
vocab = [v for v in vocab if len(v) > MIN_CHAR_LEN_IDENTITIES]

print(len(vocab))

# remove the number indicator on its own
with open('vocab_{}.txt'.format(filename).replace(".tsv",""), 'w') as f:
    f.write('\n'.join(vocab))

444149


In [36]:
fcall = partial(find_identifiers_simple)
results = Parallel(n_jobs=16)(delayed(fcall)(d) for d in tqdm_notebook(df.description))

  0%|          | 0/517827 [00:00<?, ?it/s]

In [37]:
with open("output_{}.tsv".format(filename.replace(".tsv","")), "w") as of:
    of.write("{}\n".format("\t".join(['uid','term','position','n_type','n_total','termtype'])))
    id_field = 'uid' if 'uid' in df.columns else 'id_str'
    for i, (identities, emojis) in enumerate(results):
        if i % 200000 == 0: 
            print(i)
        uid = df.iloc[i][id_field]
        idlen = str(len(identities))
        emoji_len = str(len(emojis))
        tot_len = str(len(identities) + len(emojis))
        for j,identity in enumerate(identities):
            of.write("{}\n".format("\t".join([uid,identity,str(j),idlen,tot_len,"identity"])))
        
        for j,emoji in enumerate(emojis):
            of.write("{}\n".format("\t".join([uid,emoji,str(j),idlen,tot_len,"emoji"])))    

0
200000
400000


In [ ]:

with open("lang_{}.tsv".format(filename.replace(".tsv","")), "w") as of:
        for i, row in tqdm(df.iterrows()):
            m = fmodel.predict([row['description']])  # ([['__label__en']], [array([0.9331119], dtype=float32)]
            if m and m[0] and m[0][0] and m[0][0][0]:
                of.write(str(row['uid']) + "\t" + m[0][0][0].replace("__label__","")+"\n")